# Data Cleaning

This notebook is used to clean and combine all datasources into a coherent dataset. 

We have three different data groups:
* Label & Patient Data
* Lab Data
* MRI Data

All of these have to be cleaned, selected and combined into one dataset.

## Imports

In [ ]:
import pandas as pd
import datetime
import numpy as np
import re

# Data Cleaning and Selection Label & Patient-data
First we will clean the data for our labels and patients. This includes the basic cleaning and data type matching as well as looking at anomalies and define the output format. We will use the prepared 'no duplicate PID' Sheet with Labels from KSA.

In [ ]:
print("Start Clean and Preprocessing patients-data")

In [ ]:
df_patients = pd.read_excel(r'../raw_data/Hypophysenpatienten.xlsx',sheet_name='no duplicate PID')
df_patients.head()

In [ ]:
df_patients.tail()

## Basic Cleaning, Column Selection, Anomaly Correction and Format definition


### Column Selection
First we select only the columns which have value to our model or our analysis. Some columns are already renamed to make their content more intuitive.

In [ ]:
# define needed columns
column_list = ['PID','Fall Nr.',"Datum/Zeit","Arbeitsplatz.Kürzel",'Grösse',
       'Ausfälle prä', 'Qualität', 'ED','OP Datum',
       'Diagnose', 'Kategorie', 'Patient Alter',
       'Prolaktin',"IGF1", 'Cortisol','fT4','weiteres Labor','Gender']
df_patients = df_patients[column_list]
# rename columns
df_patients= df_patients.rename(columns={"Fall Nr.": "Case_ID","PID": "Patient_ID",
                       "Datum/Zeit": "Date_Case","ED": "Diagnosis_Date", "OP Datum": "Operation_date",
                       "Arbeitsplatz.Kürzel":"ID_MRI_Machine","Grösse": "Adenoma_size","Qualität": "Label_Quality",
                       "Patient Alter":"Patient_age","Kategorie":"Category","Diagnose":"Diagnosis",
                       "Prolaktin":"Prolactin","weiteres Labor":"Lab_additional", 'Gender':"Patient_gender"})

### Check for Anomalies and correct them
There are some Anomalies mostly in the datetime columns (eg. Operation date before Entry Date). These are corrected or were corrected by the KSA after feedback from us. 

In [ ]:
# rows where Entry Date is after Operationdate?
assert len(df_patients[df_patients['Operation_date'] < df_patients['Diagnosis_Date']][['Diagnosis_Date','Operation_date']]) ==0

### Data Type Definition
Now we check the column data-types and parse them into their resprective type if not already correct.


In [ ]:
# make datetime values
df_patients["Date_Case"] = pd.to_datetime(df_patients["Date_Case"])
df_patients["Diagnosis_Date"] = pd.to_datetime(df_patients["Diagnosis_Date"])
df_patients["Operation_date"] = pd.to_datetime(df_patients["Operation_date"])

In [ ]:
# set category data type in pandas, check datatypes
df_patients['ID_MRI_Machine'] = df_patients['ID_MRI_Machine'].astype('category')
df_patients['Adenoma_size'] = df_patients['Adenoma_size'].astype('category')
df_patients['Diagnosis'] = df_patients['Diagnosis'].astype('category')
df_patients['Category'] = df_patients['Category'].astype('category')
df_patients['Patient_gender'] = df_patients['Patient_gender'].astype('category')
df_patients.dtypes

### Check Duplicates
Check if a patient is duplicated.

In [ ]:
# Patient ID Duplicate Check
assert len(df_patients[df_patients["Patient_ID"].duplicated()]) == 0

## One Hot Encode Categorical Values

To use and analyse the categorical data we need to one-hot encode them. This is done by splitting the comma separated strings into single strings and then create a one-hot-encoded column of each individual value. This column is then added to the original dataframe.

In [ ]:
df_patients["Ausfälle prä"]= df_patients["Ausfälle prä"].str.replace(' ', '')
df_patients["Ausfälle prä"]= df_patients["Ausfälle prä"].str.lower()
# Split the 'Ausfälle prä' column into separate strings
df_patients['Ausfälle prä'] = df_patients['Ausfälle prä'].str.split(',')

# Create a set to store all unique disfunctions
unique_disfunctions = set()

# Iterate over the 'Ausfälle prä' column to gather unique disfunctions
for value in df_patients['Ausfälle prä']:
    if isinstance(value, list):
        unique_disfunctions.update(value)
    elif isinstance(value, str):
        unique_disfunctions.add(value)

# Iterate over the unique disfunctions and create one-hot encoded columns
for disfunction in unique_disfunctions:
    df_patients["Pre_OP_hormone_"+ disfunction] = df_patients['Ausfälle prä'].apply(lambda x: 1 if (isinstance(x, list) and disfunction in x) or (x == disfunction) else 0)
# drop the original 'Ausfälle prä' column
df_patients = df_patients.drop('Ausfälle prä', axis=1)

## Remove All NA and not needed Labels

In [ ]:
# remove all labels which are not prolaktion or non-prolaktinom
df_patients = df_patients[df_patients['Category'].isin(['non-prolaktinom','prolaktinom'])]
assert len(df_patients['Category'].unique()) == 2

In [ ]:
df_patients.to_csv(r'../raw_data/label_data.csv',index=False)

In [ ]:
print("End Clean and Preprocessing patient data")

# Data Cleaning and Selection MRI-data

Now we will clean all MRI's.

In [ ]:
print("Start Clean and Preprocessing mri data")

### Column Selection
Only select the interesting columns for the mri's.

In [ ]:
# read MRI Data
df_mri = pd.read_excel(r'../raw_data/Hypophysenpatienten.xlsx',sheet_name='w duplicates')
# select and rename columns
column_list_mri = ['PID','Fall Nr.',"Datum/Zeit","Arbeitsplatz.Kürzel",'%ID']
df_mri = df_mri[column_list_mri]
df_mri= df_mri.rename(columns={"Fall Nr.": "Case_ID","PID": "Patient_ID",
                       "Datum/Zeit": "Date_Case","Arbeitsplatz.Kürzel":"ID_MRI_Machine",'%ID':"MRI_Case_ID",})

# replace chars and make a senseful caseid
df_mri['Case_ID'] = df_mri['Case_ID'].replace('-', '', regex=True)
df_mri['Case_ID'] = df_mri['Case_ID'].astype(int)

In [ ]:
# merge patient data to mri
df_mri_merge = df_mri.merge(df_patients[['Patient_ID','Operation_date','Diagnosis_Date']], how='left', on="Patient_ID")
# create op and non op patient mri subsets
df_mri_op = df_mri_merge[~df_mri_merge['Operation_date'].isna()]
df_mri_nonop = df_mri_merge[df_mri_merge['Operation_date'].isna()]

# for patients with an operation get newest MRI, which is not newer than the operation date and not older than the diagnosis date 
df_mri_op = df_mri_op[(df_mri_op['Date_Case'] <= df_mri_op['Operation_date']) &  (df_mri_op['Date_Case'] >= df_mri_op['Diagnosis_Date'])].groupby(["Patient_ID","Case_ID"]).min().reset_index()
# for patients without an operation get oldest MRI, which is not older than the diagnosis date
df_mri_nonop = df_mri_nonop[(df_mri_nonop['Date_Case'] >= df_mri_nonop['Diagnosis_Date'])].groupby(["Patient_ID","Case_ID"]).min().reset_index()
# concat subsets
df_mri = pd.concat([df_mri_op,df_mri_nonop]).reset_index(drop=True)
# remove same day multiples
n_cases = len(df_mri)
df_mri_clean = df_mri.groupby(["Patient_ID","Case_ID"])[["Date_Case",'ID_MRI_Machine',"MRI_Case_ID"]].min().reset_index()

# if there are multiple
print(f"{n_cases-len(df_mri_clean)} Cases were deleted, because they were same-day duplicates.")

In [ ]:
# There are no MRI-files for these Cases. [8152562, 7661718, 7371681]
no_mri_cases = [41040108, 40632245,41040108,40367289, 40089238]
df_mri_clean = df_mri_clean[~df_mri_clean["Case_ID"].isin(no_mri_cases)]

In [ ]:
df_mri_clean = df_mri_clean.sort_values('Patient_ID')
df_mri_clean.to_csv(r'../raw_data/mri_data_all.csv',index=False)

In [ ]:
print("End Clean and Preprocessing mri data")

# Data Cleaning and Selection Lab-data

Now the lab data from the explicit KSA export will be cleaned.

In [ ]:
print("Start Clean and Preprocessing lab-data")

### Read

In [ ]:
lab_data = pd.read_excel("../raw_data/extract_pit.xlsx",
                         usecols=['PATIENT_NR','FALL_NR','Analyse-ID','Resultat','Datum_Resultat','Auftragsdatum']).rename(
                             columns={"PATIENT_NR":"Patient_ID","FALL_NR":"Case_ID","Analyse-ID":"Lab_ID",})

### Clean and select Lab's
There is a multitude of labs in the export. We do not need all of them. 

In [ ]:
# remove not needed labs
lab_data = lab_data[~lab_data['Lab_ID'].isin(['ABTEST','TBILHB'])].copy()

In [ ]:
# rename labs which are integer based with a string name
lab_data['Lab_ID'] = lab_data['Lab_ID'].replace({20396:'IGF1',24382:'PROL',24384:'PROL',24383:'PROL',
                                                 'COR30':'COR','COR60':'COR',
                                                 'CORL0':'COR','CORL30':'COR',
                                                 'COR0':'COR'})

In [ ]:
lab_data['Auftragsdatum'] = pd.to_datetime(lab_data['Auftragsdatum'], format='%Y%m%d')

In [ ]:
# replace some not used characters in the case ids
lab_data['Case_ID'] = lab_data['Case_ID'].replace('#','',regex=True)
lab_data['Case_ID'] = lab_data['Case_ID'].astype(int)

In [ ]:
# clean result column
lab_data['Resultat'] = lab_data['Resultat'].replace(',','.',regex=True)
lab_data['Resultat'] = lab_data['Resultat'].replace('>','',regex=True)
lab_data['Resultat'] = lab_data['Resultat'].replace('<','',regex=True)
lab_data['Resultat'] = lab_data['Resultat'].replace('¬†','',regex=True)
lab_data['Resultat'] = lab_data['Resultat'].astype(float)

In [ ]:
# replace export anomalies 
ids = {'Ã¼': 'ü', 'Ã¤': 'ä', "Ã„":"Ä","√§":"ä"}

for column in lab_data.columns[lab_data.columns.isin(["Case_ID","Patient_ID","Datum_Resultat","Auftragsdatum"]) == False]:
    for old, new in ids.items():
        lab_data[column] = lab_data[column].replace(old, new, regex=False)

# clean the greather and less than characters with regex
clean_result = lambda result: re.sub(r'(?<!\d)\.', '', re.sub(r'[^\d.]', '', str(result))) #clean < zahl / > zahl / 1 A zahl
lab_data["Resultat"] = lab_data["Resultat"].apply(clean_result) 
# remove empty results
lab_data = lab_data[lab_data["Resultat"] != ""]
lab_data["Resultat"] = lab_data["Resultat"].astype(float)

In [ ]:
# check if the datetime was correctly fixed
assert lab_data["Datum_Resultat"].min() > pd.to_datetime("1995-01-01")
assert lab_data["Auftragsdatum"].min() > pd.to_datetime("1995-01-01")

In [ ]:
# mean of results of same date
lab_data = lab_data.groupby(["Patient_ID",'Case_ID',"Lab_ID","Auftragsdatum"])["Resultat"].agg(['mean']).reset_index()

In [ ]:
# make dataframe wide
lab_data = lab_data.pivot(index=["Patient_ID","Case_ID","Auftragsdatum"], values = ['mean'], columns = ['Lab_ID'])
lab_data.columns = lab_data.columns.droplevel()
lab_data = lab_data.reset_index()

### Create LabData from label data

In [ ]:
df_additional_lab = pd.read_csv(r'../raw_data/label_data.csv').rename(columns={'Cortisol':'COR','fT4':'FT4','Prolactin':'PROL'})[['Patient_ID','Case_ID','COR','FT4','PROL','IGF1','Lab_additional']]
df_additional_lab = df_additional_lab.dropna(subset=['PROL','IGF1','COR','FT4','Lab_additional'], how='all').reset_index(drop=True)

In [ ]:
df_additional_lab['Lab_additional'] = df_additional_lab['Lab_additional'].fillna('')
for i in ['Test', 'LH','FSH']:
    df_additional_lab[i] = ''
    indices = df_additional_lab[df_additional_lab['Lab_additional'].str.contains(i)].index
    df_additional_lab.loc[indices,i] = df_additional_lab.iloc[indices]['Lab_additional']

In [ ]:
df_additional_lab = df_additional_lab.drop(columns=['Lab_additional'])
df_additional_lab = df_additional_lab.rename(columns={'Test':'TEST'})

#### Testosteron Cleaning

In [ ]:
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r' nmol/l','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'nmol/l','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'nmol','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'Testo','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'Test','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r',','.',regex=True)
df_additional_lab.loc[df_additional_lab['TEST'] == '', 'TEST'] = np.nan
df_additional_lab['TEST']= df_additional_lab['TEST'].astype(float)

#### LH Cleaning

In [ ]:
df_additional_lab.loc[df_additional_lab['LH'] == '', 'LH'] = np.nan
df_additional_lab['LH'] = df_additional_lab['LH'].replace(r'FSH \d*U\/L,','',regex=True)
df_additional_lab['LH'] = df_additional_lab['LH'].replace(r'LH','',regex=True)
df_additional_lab['LH'] = df_additional_lab['LH'].replace(r'U/L','',regex=True)
df_additional_lab['LH']= df_additional_lab['LH'].astype(float)

#### FSH Cleaning

In [ ]:
df_additional_lab['FSH'] = df_additional_lab['FSH'].replace(r'FSH','',regex=True)
df_additional_lab['FSH'] = df_additional_lab['FSH'].replace(r'U/L','',regex=True)
df_additional_lab['FSH'] = df_additional_lab['FSH'].replace(r', LH \d*','',regex=True)

df_additional_lab.loc[df_additional_lab['FSH'] == '', 'FSH'] = np.nan
df_additional_lab['FSH']= df_additional_lab['FSH'].astype(float)

#### Cortisol Cleaning 

In [ ]:
df_additional_lab['COR'] = df_additional_lab['COR'].replace(r' nmol/l','',regex=True)
df_additional_lab['COR']= df_additional_lab['COR'].astype(float)

#### FT4 Cleaning

In [ ]:
df_additional_lab['FT4'] = df_additional_lab['FT4'].replace(r' pmol/l','',regex=True)
df_additional_lab['FT4']= df_additional_lab['FT4'].astype(float)

#### Prolaktin Cleaning and Conversion

In [ ]:
df_additional_lab["PROL"]= df_additional_lab["PROL"].str.replace("ug/L","ug/l")
df_additional_lab["PROL"]= df_additional_lab["PROL"].str.replace("mu/L","mU/l")

In [ ]:
# get indices which need to be converted
# indices_to_divide = df_additional_lab.loc[df_additional_lab["PROL"].str.contains('mU/l'),'PROL'].index 
indices_to_divide = df_additional_lab[~df_additional_lab["PROL"].isna() & df_additional_lab['PROL'].str.contains('mU/l')].index 
# remove units and strings
df_additional_lab['PROL'] = df_additional_lab['PROL'].str.rstrip(r'mU/l')
df_additional_lab['PROL'] = df_additional_lab['PROL'].str.rstrip(r'ug/l')
df_additional_lab['PROL'] = df_additional_lab['PROL'].str.rstrip(r'ug/L')
df_additional_lab['PROL'] = df_additional_lab['PROL'].astype(float)
# mU/l -> ug/l (mU/l * 0.048)
df_additional_lab.loc[indices_to_divide,'PROL'] = df_additional_lab.loc[indices_to_divide,'PROL'] * 0.048 

#### IGF1 Cleaning and Conversion

In [ ]:
# df_additional_lab["IGF1"]= df_additional_lab["IGF1"].str.replace("ug/L","ug/l")
df_additional_lab["IGF1"]= df_additional_lab["IGF1"].str.replace("ug/l","")
df_additional_lab["IGF1"]= df_additional_lab["IGF1"].str.replace(",",".")

# get indices which need to be converted
# indices_to_divide = df_additional_lab.loc[df_additional_lab["IGF1"].str.contains('ng/ml'),'IGF1'].index 
indices_to_divide = df_additional_lab[~df_additional_lab["IGF1"].isna() & df_additional_lab['IGF1'].str.contains('ng/ml')].index
# remove units and strings
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].str.rstrip(r'ng/ml')
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].str.rstrip(r'nmol')
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].str.rstrip(r'nmol/l')
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].astype(float)
# ng/ml -> nmol/l (ng/ml * 0.13)
df_additional_lab.loc[indices_to_divide,'IGF1'] = df_additional_lab.loc[indices_to_divide,'IGF1'] * 0.13

In [ ]:
# combine additional lab and lab data by combining the first occurence
lab_data = lab_data.set_index(['Patient_ID','Case_ID']).combine_first(df_additional_lab.set_index(['Patient_ID','Case_ID'])).reset_index()

In [ ]:
# merge patient data to labdata
lab_data_merge = lab_data.merge(df_patients[['Patient_ID','Operation_date','Diagnosis_Date']], how='left', on="Patient_ID")

# create op and non op patient labdata subsets
lab_data_op = lab_data_merge[~lab_data_merge['Operation_date'].isna()]
lab_data_nonop = lab_data_merge[lab_data_merge['Operation_date'].isna()]
# for patients with an operation get newest labdata, which is not newer than the operation date (including an offset of 2 months, after op diagnostics is later) and not older than the diagnosis date 
lab_data_op = lab_data_op[(lab_data_op['Auftragsdatum'] <= lab_data_op['Operation_date'] + pd.DateOffset(weeks=1)) & (lab_data_op['Auftragsdatum'] >= lab_data_op['Diagnosis_Date'])].groupby(["Patient_ID","Case_ID"]).min().reset_index()
# for patients without an operation get oldest labdata, which is not older than the diagnosis date
lab_data_nonop = lab_data_nonop[(lab_data_nonop['Auftragsdatum'] >= lab_data_nonop['Diagnosis_Date'])].groupby(["Patient_ID","Case_ID"]).min().reset_index()


# concat subsets
lab_data = pd.concat([lab_data_op,lab_data_nonop]).reset_index(drop=True)

# add more patient information
lab_data = lab_data.merge(df_patients[['Patient_ID','Patient_gender','Patient_age','Date_Case','Adenoma_size',
                                       ]+[col for col in df_patients.columns if "Pre_" in col]], how='left', on="Patient_ID")
# remove same day multiples
n_cases = len(lab_data)
lab_data_clean = lab_data.groupby(["Patient_ID","Case_ID"])[['Auftragsdatum','COR', 'FSH', 'FT4', 'IGF1',
       'LH', 'PROL', 'TEST',]].min().reset_index()

# if there are multiple
print(f"{n_cases-len(lab_data_clean)} Cases were deleted, because they were same-day duplicates.")

In [ ]:
spar = round(lab_data_clean[['COR', 'FSH','LH', 'FT4', 'IGF1', 'LH', 'PROL','TEST']].isna().mean().mean(),3)
print(f"Sparsity of labordata: {spar} %")

In [ ]:
lab_data_clean = lab_data_clean.sort_values('Patient_ID')
lab_data_clean.to_csv(r'../raw_data/lab_data_all.csv',index=False)

In [ ]:
print("End Clean and Preprocessing labor data")

# Full Merge

In [ ]:
# get mridata where an mri matches to with case and patient if of a lab
df_temp = pd.merge(df_mri_clean,lab_data_clean,left_on=['Patient_ID','Case_ID'],right_on=['Patient_ID','Case_ID'])

In [ ]:
# match this data with the patients data
full_merged = pd.merge(df_temp,df_patients[['Patient_ID','Category','Patient_gender','Patient_age','Adenoma_size']+
                                           list(df_patients.columns[df_patients.columns.str.contains('Pre')])+
                                           ['Operation_date', 'Diagnosis_Date']]
                                           ,how='inner',left_on=['Patient_ID'],right_on=['Patient_ID'])

In [ ]:
full_merged = full_merged.sort_values('Patient_ID')

In [ ]:
full_merged["label"] = (full_merged["Category"]=="prolaktinom").astype(int)

In [ ]:
assert full_merged.duplicated(subset=['Patient_ID','Case_ID']).sum() == 0

In [ ]:
full_merged.to_csv(r'../data/test/test_data_pairs.csv',index=False)

In [ ]:
# remove all non pair data
labdata_only_dirty = lab_data[~lab_data['Case_ID'].isin(full_merged['Case_ID'])]
mri_only_dirty = df_mri_clean[~df_mri_clean['Case_ID'].isin(full_merged['Case_ID'])]

# add nan as fold
labdata_only_dirty.to_csv(r'../raw_data/lab_data_only_dirty.csv',index=False)
mri_only_dirty.to_csv(r'../raw_data/mri_data_only_dirty.csv',index=False)

In [ ]:
print(f"There are {len(df_mri_clean)} MRI's with all constraints applied!")
print(f"There are {len(lab_data_clean)} Lab's with all constraints applied!")
print(f"There are {len(full_merged)} Combinations with all constraints applied!")
print(f"There are {len(labdata_only_dirty)} Lab's which are dirty!")
print(f"There are {len(mri_only_dirty)} MRI's which are dirty!")
